<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Complaints Classification using Vantage and LLM
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial'>Revolutionize customer complaint resolution with our pioneering solution, which seamlessly integrates the capabilities of <b>Teradata Vantage</b> and <b>AWS Bedrock - Anthropic's Claude LLM model</b> model as LLM. This powerful synergy enables businesses to classify customer complaints with unmatched precision and speed, allowing them to swiftly identify and address concerns, thereby elevating overall customer satisfaction and loyalty.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Key Features:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li><b>Automated Classification:</b> Our AI-driven model categorizes complaints into predefined categories, ensuring consistency and reducing manual effort.</li>
    <li><b>Contextual Understanding:</b> The system comprehends the nuances of customer feedback, capturing subtle differences in tone and language.</li>
    <li><b>Real-time Insights:</b> Generate instant reports and analytics to identify trends, patterns, and areas for improvement.</li>
</ul>


<p style = 'font-size:16px;font-family:Arial'><b>Benefits:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li><b>Enhanced Customer Experience:</b> Swiftly address customer concerns, fostering trust and loyalty.</li>
    <li><b>Improved Operational Efficiency:</b> Reduce manual processing time, allowing teams to focus on high-value tasks.</li>
    <li><b>Data-Driven Decision Making: </b> Make informed decisions with actionable insights from complaint data.</li>
</ul>

<p style = 'font-size:16px;font-family:Arial'>Experience the transformative power of Generative AI in complaints classification.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Configuring AWS Bedrock - Anthropic's Claude LLM model</li>
    <li>Classify Complaints</li>
    <li>Cleanup</li>
</ol>

<hr style='height:2px;border:none;'>
<b style = 'font-size:20px;font-family:Arial'>1. Configuring the environment</b>
<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Downloading and installing additional software needed</b>

In [ ]:
%%capture
!pip install -r requirements.txt --upgrade --quiet

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.2 Import the required libraries</b></p>
<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Data manipulation and analysis
import numpy as np
import pandas as pd
import json, warnings
import getpass

# Visualization
import plotly.express as px
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Progress bar
from tqdm import tqdm

# Machine learning and other utilities from Teradata
from teradataml import *
from teradatagenai import TeradataAI, TextAnalyticsAI, VSManager, VectorStore, VSApi

# Requests
import requests

# Display settings
display.max_rows = 5
pd.set_option('display.max_colwidth', None)
# Set display options for dataframes, plots, and warnings
%matplotlib inline
warnings.filterwarnings('ignore')
display.suppress_vantage_runtime_warnings = True

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
print("Checking if this environment is ready to connect to VantageCloud Lake...")

if os.path.exists("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env"):
    print("Your environment parameter file exist.  Please proceed with this use case.")
    # Load all the variables from the .env file into a dictionary
    env_vars = dotenv_values("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env")
    # Create the Context
    eng = create_context(host=env_vars.get("host"), username=env_vars.get("username"), password=env_vars.get("my_variable"))
    execute_sql('''SET query_band='DEMO=text_analytics_teradatagenai_aws_huggingface.ipynb;' UPDATE FOR SESSION;''')
    print("Connected to VantageCloud Lake with:", eng)
else:
    print("Your environment has not been prepared for connecting to VantageCloud Lake.")
    print("Please contact the support team.")

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:2px;border:none'>
<p style = 'font-size:20px;font-family:Arial'><b>2.  Set up the LLM connection</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <b>teradatagenai</b> python library can both connect to cloud-based LLM services as well as instantiate private models running <b>at scale</b> on local GPU compute. In this case we will use anthropoc claude-instant-v1 for low-cost, high-throughput tasks.</p>

<ol style = 'font-size:16px;font-family:Arial'>
<li><b>aws_access_key_id</b>: Enter your AWS access key ID</li>
<li><b>aws_secret_access_key</b>: Enter your AWS secret access key</li>
<li><b>region name</b>: Enter the AWS region you want to configure (e.g., us-east-1)</li>
<ol>

In [ ]:
access_key = getpass.getpass('aws_access_key_id: ')
secret_key = getpass.getpass('aws_secret_access_key: ')
region_name = getpass.getpass('region name: ')

<hr style='height:2px;border:none'>
<p style='font-size:20px;font-family:Arial'><b>3. Use the <code>TextAnalyticsAI</code> API to Perform Various Text Analytics Tasks</b></p>
<p style='font-size:16px;font-family:Arial'>You can execute the help function at the bottom of this notebook to read more about this API.</p>

In [ ]:
# Provide model details
model_name="us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Select in-database or external model
llm = TeradataAI(api_type = 'AWS',
         model_name = model_name,
         region = region_name,
         # authorization = 'Repositories.BedrockAuth'
         access_key = access_key,
         secret_key = secret_key)

obj = TextAnalyticsAI(llm=llm)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Classify Complaints</b>
<p style = 'font-size:16px;font-family:Arial'>We'll use a sample of the data to classify complaints</p>

In [ ]:
tdf = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Consumer_Complaints'))
tdf = tdf.assign(id = tdf.complaint_id).drop('complaint_id', axis = 1)

In [ ]:
tdf_sample = tdf.sample(100)

In [ ]:
tdf_classified = obj.classify(
    column="consumer_complaint_narrative",
    data=tdf_sample,
    accumulate="0:5",
    labels=["Complaint", "Non-Complaint"],
    multi_label=True
)

In [ ]:
tdf_classified = tdf_classified.assign(Prediction = tdf_classified.Labels.oreplace("[").oreplace("]")).drop(columns=['Labels', 'Message'])
tdf_classified = tdf_classified.assign(Prediction = tdf_classified.Prediction.cast(type_=VARCHAR(15)))
tdf_classified = tdf_classified.assign(Prediction = tdf_classified.Prediction.str.strip())

In [ ]:
tdf_classified

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.1 Consumer Complaints Prediction vs Occurrences</b></p>

<p style = 'font-size:16px;font-family:Arial'>A graph illustrating the relationship between consumer complaints prediction and the number of occurrences. This visual representation helps identify trends, patterns, and areas for improvement, enabling data-driven decision making.</p>

In [ ]:
from IPython.display import display, Markdown
def display_helper(msg):
    return display(Markdown(
        f"""<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'><b>Note: </b>
<i>{msg}</i></p>"""))

In [ ]:
from collections import Counter
import plotly.io as pio
pio.renderers.default = "notebook_connected"

df = tdf_classified.to_pandas()
data = Counter(df['Prediction'])

# Convert Counter data to DataFrame
viz_df = pd.DataFrame.from_dict(data, orient='index', columns=['Count']).reset_index()

# Rename columns
viz_df.columns = ['Prediction', 'Count']

# Create bar graph using Plotly Express
fig = px.bar(viz_df, x='Prediction', y='Count', color='Prediction',
             labels={'Count': 'Number of Occurrences', 'Prediction': 'Prediction'})

# Show the plot
fig.show()

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.2 Word Cloud for Consumer Complaints Prediction</b></p>

<p style = 'font-size:16px;font-family:Arial'>A visual representation of <b>consumer complaints prediction</b>, highlighting the most frequent words and pain points in customer feedback. This word cloud helps identify trends, sentiment, and areas for improvement, enabling data-driven decision making.</p>

In [ ]:
complaint = df[df['Prediction'] == 'Complaint']
complaint_text = ' '.join(complaint['consumer_complaint_narrative'])

# Replace 'X' with blank space
modified_string = complaint_text.replace('X', '')

if len(modified_string) > 0:
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(modified_string)

    # Display the word cloud
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title("Complaints")
    plt.tight_layout()
    plt.axis("off")
    plt.show()
else:
    display_helper("We included both complaint and non-complaint options for completeness. But since this is a complaints dataset, we don't expect to see any complaints.")

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.3 Word Cloud for Non-Complaints Prediction</b></p>

<p style = 'font-size:16px;font-family:Arial'>A visual representation of <b>non-complaints prediction</b>, highlighting the most frequent words and positive sentiments in customer feedback. This word cloud helps identify trends, sentiment, and areas of satisfaction, enabling data-driven decision making.</p>

In [ ]:
non_complaint = df[df['Prediction'] == 'Non-Complaint']
non_complaint_text = ' '.join(non_complaint['consumer_complaint_narrative'])

# Replace 'X' with blank space
modified_string = non_complaint_text.replace('X', '')

if len(modified_string) > 0:
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(modified_string)

    # Display the word cloud
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title("Non-Complaints")
    plt.tight_layout()
    plt.axis("off")
    plt.show()
else:
    display_helper("We included both complaint and non-complaint options for completeness. But since this is a complaints dataset, we don't expect to see any non-complaints.")

<p style = 'font-size:16px;font-family:Arial'>Now the results can be saved back to Vantage.</p>

In [ ]:
copy_to_sql(df = df, table_name = 'complaints_classified', if_exists = 'replace')

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>5. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
tables = ['complaints_classified']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name=table)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024, 2025, 2026. All Rights Reserved
        </div>
    </div>
</footer>